In [1]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession.builder\
        .master("spark://HTPC.:7077") \
        .appName("Emil_winder")\
        .config("spark.cores.max", 4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

23/03/10 08:54:33 WARN Utils: Your hostname, HTPC resolves to a loopback address: 127.0.1.1; using 172.25.222.32 instead (on interface eth0)
23/03/10 08:54:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/10 08:54:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# A.1.1
spark_context = spark_session.sparkContext
df = spark_session.read.json("hdfs://localhost:9000/user/emwind/input/sample_data.json")


In [3]:
df.count()

10000

In [4]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)



In [5]:
df.select("body").show()

+--------------------+
|                body|
+--------------------+
|            A quarry|
|[Salutations! I'm...|
|I got into baseba...|
|        FUCKING TORY|
|I see a water dra...|
|Wait. The Michiga...|
|              ye fam|
|143417804| &gt; U...|
|That is some chic...|
|Does he even know...|
|            Tequila.|
|your heart beats ...|
|&gt; Subscribe: /...|
|you're really ign...|
|lets see how deep...|
|You are arguing t...|
|I'm thinking abou...|
|[Original post](h...|
|I think that's a ...|
|Harp absolutelly....|
+--------------------+
only showing top 20 rows



In [76]:
import pyspark.sql.functions as psf
print(df.count())
words_list = ["bitcoin", "BTC","cryptocurrency"]
df2 = df.filter(psf.col('body').rlike('(^|\s)(' + '|'.join(words_list) + ')(\s|$)')).select("body")

10000


In [11]:
df2.count()

18

In [47]:
from pyspark.sql.functions import col
df2.where(col("extracted_word") != None).select("extracted_word").count()

0

In [77]:
import pyspark.sql.functions as F
keywords = ["bitcoin", "btc", "cryptocurrency"]
re_pattern = '(' + '|'.join([fr'\\b{k}\\b' for k in keywords]) + ')'
df3 = df.withColumn('matched', F.expr(f"regexp_extract_all(body, '(?i){re_pattern}', 1)"))
df3 = df3.withColumn('count', F.size('matched'))


In [78]:
from pyspark.sql.functions import col
df4 = df3.filter(col('count') != 0)

In [79]:
df4.count()

26

In [80]:
df4 = df4.select('count', 'body','matched')
df4.show()

+-----+--------------------+--------------------+
|count|                body|             matched|
+-----+--------------------+--------------------+
|    3|The title, it is ...|[bitcoin, bitcoin...|
|    1|What do you think...|               [btc]|
|    2|The problem is th...|  [Bitcoin, Bitcoin]|
|    1|I’m really intere...|    [cryptocurrency]|
|    3|I like to diversi...| [BTC, BTC, Bitcoin]|
|    1|     Lets bet 1 btc?|               [btc]|
|    2|Asicboost does no...|  [bitcoin, bitcoin]|
|    1|If we see an esca...|           [Bitcoin]|
|    2|I don't see the c...|      [btc, bitcoin]|
|    1|Amen. Hoping Bitc...|           [Bitcoin]|
|    1|Ok you should rea...|           [bitcoin]|
|    1|These people are ...|               [BTC]|
|    2|I think it is a s...|  [Bitcoin, Bitcoin]|
|    1|I wouldn't call [...|               [BTC]|
|    3|So I've been read...|[bitcoin, Bitcoin...|
|    1|Here is the post ...|               [BTC]|
|    5|u/jessquit, you'v...|[Bitcoin, bitcoin...|


In [71]:
dff = spark_session.read.json("hdfs://localhost:9000/user/emwind/2013-11.json")

In [74]:
df3 = dff.withColumn('matched', F.expr(f"regexp_extract_all(body, '(?i){re_pattern}', 1)"))
df3 = df3.withColumn('count', F.size('matched'))
df5 = df3.filter(col('count') != 0)

In [75]:
df5.count()

88858

In [81]:
dff.count()

37396497

In [82]:
print(26/10000,88858/37396497 )

0.0026 0.002376104906296437
